# Solutions I:  Exploration

### Related to client:

1. `age`: age in years (numeric)
2. `job`: type of job (categorical)
3. `marital` : marital status (categorical)
4. `education`: level of education(categorical)
5. `default`: has credit in default? (categorical)
6. `housing`: has housing loan? (categorical)
7. `loan`: has personal loan? (categorical)

### Related to the last contact of the current campaign:

8. `contact`: contact communication type (categorical)
9. `month`: last contact month of year (categorical)
10. `day_of_week`: last contact day of the week (categorical)
11. `duration`: last contact duration, in seconds (numeric)

*Note: `duration` is not known prior to contacting the client; so `duration = 0` means
`y = 0`. Therefore `duration` should not be included for prediction.*

### Other attributes:

12. `campaign`: number of contacts performed during this campaign and for this client (numeric)
13. `pdays`: number of days passed after the client was last contacted for a previous campaign (numeric)
14. `previous`: number of contacts performed before this campaign and for this client (numeric)
15. `poutcome`: outcome of the previous marketing campaign (categorical)

### Social and economic context:

16. `emp.var.rate`: employment variation rate - quarterly indicator (numeric)
17. `cons.price.idx`: consumer price index - monthly indicator (numeric)
18. `cons.conf.idx`: consumer confidence index - monthly indicator (numeric)
19. `euribor3m`: euribor 3 month rate - daily indicator (numeric)
20. `nr.employed`: number of employees - quarterly indicator (numeric)

In [15]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Load dataset

In [ ]:
df = pd.read_csv("../../0_data/banking/bank-additional-full.csv", sep=";")
df.shape

In [ ]:
# Columns in the dataset
df.columns

In [ ]:
df.dtypes

In [ ]:
# NUmber of missing values
df.isna().sum()

## Basic descriptives

In [ ]:
# Target variable distribution
# Note: Quite skewed only ~1 in 10 takes the term deposit.
df["y"].value_counts()

In [ ]:
# Or as percentage...
df["y"].value_counts(normalize=True)

### Numerical features

In [22]:
numerical = df.select_dtypes("number")

In [ ]:
# Numerical features, note that:
# - age 17 - 98 (quite extreme).
# - duration is very skewed (but dropped anyway).
# - campaign is also very skewed.
# - pdays - lots of 999 values (missings).
# - previous - mostly zeroes.
numerical.describe()

In [ ]:
# Set 999 to missing for pdays
numerical["pdays"] = numerical["pdays"].replace({999: np.nan})

# Mostly missing values...
(numerical["pdays"].value_counts(dropna=False, normalize=True) * 100).head(10)

In [ ]:
numerical["previous"].nunique()

In [ ]:
# Plot numeric distributions.
# Note: Macro-economic variables have lots of repeated values.
ncols = 3
nrows = math.ceil(numerical.shape[1] / ncols)
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 3, nrows * 2))

for idx, col_name in enumerate(numerical.columns):
    # Compute location in axes grid.
    row = idx // ncols
    col = idx % ncols
    ax = axes[row, col]

    # Guestimate number of bins.
    bins = numerical[col_name].nunique()
    bins = bins if bins < 30 else math.ceil(bins / 5)
    bins = bins if bins < 50 else 50

    # Draw the histogram.
    ax.hist(numerical[col_name], bins=bins, edgecolor="white")
    ax.set_title(col_name)

plt.rc('font', size=7)
plt.tight_layout()

In [ ]:
# Correlations with target variable.
(
    numerical
    .corrwith((df["y"] == "yes").astype(int))
    .rename("Correlation with target")
    .sort_values(ascending=False)
    .round(2)
    .to_frame()
)

### Categorical

In [27]:
categorical = df.select_dtypes("object")

In [ ]:
# Categorical features.
categorical.describe()

In [ ]:
# Plot categorical distributions
ncols = 3
nrows = math.ceil(categorical.shape[1] / ncols)
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 3, nrows * 2))

for idx, col_name in enumerate(categorical.columns):
    # Compute location in axes grid
    row = idx // ncols
    col = idx % ncols
    ax = axes[row, col]

    (
        categorical
        .pivot_table(index=[col_name], columns="y", aggfunc="size", fill_value=0)
        .sort_values("no")
        .plot.barh(ax=ax)
    )

    ax.set_title(col_name)
    ax.legend(loc="lower right")

plt.rc('font', size=7)
plt.tight_layout()

In [ ]:
# Value counts for job.
categorical["job"].value_counts()